In [ ]:
# prompt: je veux importer un zip depuis mon ordi

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
import zipfile
with zipfile.ZipFile(fn, 'r') as zip_ref:
    zip_ref.extractall('/content')


Saving nlp_assignment.zip to nlp_assignment.zip
User uploaded file "nlp_assignment.zip" with length 70629 bytes


In [2]:
from enum import auto
# prompt: ouvre " /content/nlp_assignment/data/traindata.csv "

import pandas as pd

# Try to specify the delimiter explicitly. If the delimiter is a tab, use '\t'
df = pd.read_csv('/content/nlp_assignment/data/traindata.csv', delimiter='\t', on_bad_lines='skip',
                header=None, names=['label', 'catégorie', 'heure', 'origin', 'texte'])
df.head()


,label,catégorie,heure,origin,texte
0,positive,AMBIENCE#GENERAL,seating,18:25,short and sweet – seating is great:it's romant...
1,positive,AMBIENCE#GENERAL,trattoria,25:34,This quaint and romantic trattoria is at the t...
2,positive,FOOD#QUALITY,food,98:102,The have over 100 different beers to offer thi...
3,negative,SERVICE#GENERAL,STAFF,5:10,THIS STAFF SHOULD BE FIRED.
4,positive,FOOD#STYLE_OPTIONS,menu,4:8,"The menu looked great, and the waiter was very..."


In [5]:
#! pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import numpy as np
import os
from collections import Counter

from transformers import RobertaConfig, RobertaForSequenceClassification
from torch.nn import CrossEntropyLoss

class RobertaForWeightedClassification(RobertaForSequenceClassification):
    def __init__(self, config, class_weights=None):
        super().__init__(config)
        if class_weights is not None:
            self.class_weights = class_weights
            self.loss_fct = CrossEntropyLoss(weight=self.class_weights)
        else:
            self.loss_fct = CrossEntropyLoss()

    def forward(self, **kwargs):
        # Supprimer les arguments inattendus
        if "num_items_in_batch" in kwargs:
            kwargs.pop("num_items_in_batch")

        outputs = super().forward(**kwargs)

        if "labels" in kwargs:
            loss = self.loss_fct(outputs.logits, kwargs["labels"])
            return (loss, outputs.logits)
        return outputs



#pip install transformers datasets
# Définir le token HF
HF_TOKEN = "XXXXX"
os.environ["WANDB_MODE"] = "disabled"

# Fusion des colonnes pour l'entrée texte
df['input'] = df['catégorie'].astype(str) + ' : ' + df['texte'].astype(str)

# Suppression des lignes incomplètes
df = df.dropna(subset=['input', 'label'])

# Encodage des labels
labels = df['label'].unique()
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}
df['label'] = df['label'].map(label2id)

# Séparer les données en train/test sans sklearn
train_df = df[:int(0.8 * len(df))]
test_df = df[int(0.8 * len(df)):]


# Calcul des poids inverses de fréquence
label_counts = Counter(train_df['label'])
total_count = sum(label_counts.values())
class_weights = [total_count / label_counts[i] for i in range(len(labels))]
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)



# Convertir en Dataset
train_ds = Dataset.from_pandas(train_df[['input', 'label']])
test_ds = Dataset.from_pandas(test_df[['input', 'label']])

# 2. Tokenization
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", token=HF_TOKEN)

def tokenize(example):
    tokens = tokenizer(
        example["input"],
        truncation=True,
        padding="max_length",
        return_attention_mask=True  # S'assure que l'attention mask est présent
    )
    tokens["label"] = example["label"]
    return tokens


train_ds = train_ds.map(tokenize, batched=False)
test_ds = test_ds.map(tokenize, batched=False)

# Suppression de la colonne brute "input"
train_ds = train_ds.remove_columns(["input"])
test_ds = test_ds.remove_columns(["input"])

# Renommer la colonne pour qu'elle corresponde aux attentes du Trainer et du data collator
train_ds = train_ds.rename_column("label", "labels")
test_ds = test_ds.rename_column("label", "labels")

# Convertir au format torch en spécifiant explicitement les colonnes
train_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)



config = RobertaConfig.from_pretrained("roberta-base", num_labels=len(labels))
model = RobertaForWeightedClassification.from_pretrained(
    "roberta-base",
    config=config,
    token=HF_TOKEN,
    class_weights=class_weights_tensor
)

# 4. Metrics (Calcul manuel)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calcul de l'accuracy
    accuracy = np.sum(predictions == labels) / len(labels)

    # Calcul du F1 score
    unique_labels = np.unique(labels)
    f1 = 0
    for label in unique_labels:
        tp = np.sum((predictions == label) & (labels == label))
        fp = np.sum((predictions == label) & (labels != label))
        fn = np.sum((predictions != label) & (labels == label))
        if tp + fp + fn > 0:
            precision = tp / (tp + fp) if tp + fp > 0 else 0
            recall = tp / (tp + fn) if tp + fn > 0 else 0
            f1 += 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    f1 /= len(unique_labels)

    return {
        "accuracy": accuracy,
        "f1": f1,
    }
# 5. Training
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no",
    report_to=None,  # Désactiver WandB
    remove_unused_columns=False  # Désactiver la suppression des colonnes non utilisées
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
    data_collator=data_collator,  # remplace l'ancien tokenizer
)

trainer.train()


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForWeightedClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'loss_fct.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,0.390400,No log
2,1.159900,No log
3,0.808400,No log


TrainOutput(global_step=453, training_loss=0.8407089647078356, metrics={'train_runtime': 361.577, 'train_samples_per_second': 9.973, 'train_steps_per_second': 1.253, 'total_flos': 948786984327168.0, 'train_loss': 0.8407089647078356, 'epoch': 3.0})

In [9]:
datatest = pd.read_csv('/content/nlp_assignment/data/devdata.csv', delimiter='\t', on_bad_lines='skip',
                header=None, names=['label', 'catégorie', 'heure', 'origin', 'texte'])

In [13]:
import numpy as np


# Fusion des colonnes pour l'entrée texte
datatest['input'] = datatest['catégorie'].astype(str) + ' : ' + datatest['texte'].astype(str)
# Suppression des lignes incomplètes
datatest = datatest.dropna(subset=['input', 'label'])

# Encodage des labels
labels = datatest['label'].unique()
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}
datatest['label'] = datatest['label'].map(label2id)

# Créer un dataset de prédiction à partir de datatest en utilisant la colonne "input"
pred_ds = Dataset.from_pandas(datatest[['input']])
# *** The fix is here: Apply tokenizer with batched=False and then format to torch ***
def tokenize_function(examples):
    return tokenizer(examples["input"], padding="max_length", truncation=True)

# Remove this line: pred_ds = pred_ds.map(tokenizer, batched=True)
pred_ds = pred_ds.map(tokenize_function, batched=True)

# Garder uniquement les colonnes nécessaires pour le modèle
pred_ds = pred_ds.remove_columns([col for col in pred_ds.column_names if col not in ['input_ids', 'attention_mask']])
pred_ds.set_format("torch")

# Faire les prédictions avec le modèle entraîné
predictions = trainer.predict(pred_ds)

# Extraire le label prédit pour chaque exemple
pred_labels = np.argmax(predictions.predictions, axis=-1)

# Ajouter les labels prédits au DataFrame original
datatest['predicted_label'] = pred_labels

Map:   0%|          | 0/376 [00:00<?, ? examples/s]

In [19]:
# prompt: calcule les statistiques de f1 score , accuracy et recall sur datatest

from sklearn.metrics import accuracy_score, f1_score, recall_score

# Assuming 'datatest' DataFrame has 'label' (true labels) and 'predicted_label' columns
true_labels = datatest['label'].values
predicted_labels = datatest['predicted_label'].values

accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted') # Use 'weighted' for multi-class
recall = recall_score(true_labels, predicted_labels, average='weighted') # Use 'weighted' for multi-class

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Recall: {recall}")


Accuracy: 0.8563829787234043
F1 Score: 0.8470451588058273
Recall: 0.8563829787234043


In [18]:
datatest

,label,catégorie,heure,origin,texte,input,predicted_label
0,0,LOCATION#GENERAL,neighborhood,54:66,"great food, great wine list, great service in ...","LOCATION#GENERAL : great food, great wine list...",0
1,1,RESTAURANT#GENERAL,place,15:20,I thought this place was totally overrated.,RESTAURANT#GENERAL : I thought this place was ...,1
2,0,FOOD#QUALITY,Fish,0:4,Fish is so very fresh.,FOOD#QUALITY : Fish is so very fresh.,0
3,1,SERVICE#GENERAL,manager,19:26,"I showed it to the manager, and he smilingly a...","SERVICE#GENERAL : I showed it to the manager, ...",1
4,2,DRINKS#QUALITY,margaritas,63:73,"The food we ordered was excellent, although I ...",DRINKS#QUALITY : The food we ordered was excel...,0
...,...,...,...,...,...,...,...
371,0,RESTAURANT#GENERAL,PLACE,58:63,I CAN EAT HERE EVERY DAY OF THE WEEK REALLY LO...,RESTAURANT#GENERAL : I CAN EAT HERE EVERY DAY ...,0
372,2,RESTAURANT#MISCELLANEOUS,Bark,52:56,Though it's been crowded most times I've gone ...,RESTAURANT#MISCELLANEOUS : Though it's been cr...,0
373,0,FOOD#QUALITY,food,4:8,The food is excellent!,FOOD#QUALITY : The food is excellent!,0
374,1,FOOD#QUALITY,chow fun and chow see,3:24,My chow fun and chow see was really bland and ...,FOOD#QUALITY : My chow fun and chow see was re...,1


In [16]:
real = pd.read_csv('/content/nlp_assignment/data/devdata.csv', delimiter='\t', on_bad_lines='skip',
                header=None, names=['label', 'catégorie', 'heure', 'origin', 'texte'])

In [17]:
real['label'].value_counts()

,count
label,
positive,264
negative,98
neutral,14
